In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from catboost import CatBoostRegressor
import lightgbm as lgb
from feature_engineering.dataset import build_dataset
from feature_engineering.features import FEATURE_CANDIDATES, CAT_CANDIDATES, SORT_KEYS, DROP_COLS
from model.utils import metric


class Config:
    SEED = 42
    N_FOLDS = 5
    CLIP_X = (0, 105)
    CLIP_Y = (0, 68)

    # Base 모델
    BASE_PARAMS = {
        'depth' :10,
        'learning_rate' : 0.0222,
        'l2_leaf_reg' : 2.80,
        'rsm' : 0.6716,
        'bagging_temperature' : 0.121,
        'iterations' : 6000,
        'early_stopping_rounds' : 200
    }

    # Fixer 모델
    FIXER_PARAMS = {
        'depth' : 7,
        'learning_rate' : 0.0339,
        'l2_leaf_reg' : 26.74,
        'rsm' : 0.9103,
        'bagging_temperature' : 0.4276,
        'loss_function' : 'MAE',
        'early_stopping_rounds' : 250
    }

In [ ]:
# 1) CatBoost 파이프라인 (Base + LongSwap + Fixer)
#    - 반환을 voting에 쓰기 좋게 고정

def train_long_model(X_train, y_dx, y_dy, cat_features):
    # long 정의 (OOF 기준과 동일)
    pass_dist = np.sqrt(y_dx**2 + y_dy**2)
    mask_long = pass_dist >= 35

    X_long = X_train[mask_long]
    y_dx_long = y_dx[mask_long]
    y_dy_long = y_dy[mask_long]

    print(f"[LONG] samples: {len(X_long)}")

    LONG_PARAMS = {
        'depth': 7,
        'learning_rate': 0.03,
        'iterations': 1200,
        'loss_function': 'RMSE',
        'early_stopping_rounds': 100,
        'random_seed': 42,
        'verbose': 200
    }

    model_dx = CatBoostRegressor(**LONG_PARAMS)
    model_dy = CatBoostRegressor(**LONG_PARAMS)

    model_dx.fit(X_long, y_dx_long, cat_features=cat_features)
    model_dy.fit(X_long, y_dy_long, cat_features=cat_features)

    return model_dx, model_dy

def train_residual_pipeline_cat(
    X_train, y_dx, y_dy, X_test, groups, cat_features,
    long_threshold=35.0, min_long_samples=50, verbose=200
):
    gkf = GroupKFold(n_splits=Config.N_FOLDS)

    # OOF
    oof_base_dx = np.zeros(len(X_train), dtype=np.float32)
    oof_base_dy = np.zeros(len(X_train), dtype=np.float32)
    oof_fix_dx  = np.zeros(len(X_train), dtype=np.float32)
    oof_fix_dy  = np.zeros(len(X_train), dtype=np.float32)

    # TEST (분리)
    test_base_dx = np.zeros(len(X_test), dtype=np.float32)
    test_base_dy = np.zeros(len(X_test), dtype=np.float32)
    test_fix_dx  = np.zeros(len(X_test), dtype=np.float32)
    test_fix_dy  = np.zeros(len(X_test), dtype=np.float32)

    # 중요도
    fi_df = pd.DataFrame(index=X_train.columns)
    fi_df["importance"] = 0.0

    print("\n===== [CatBoost] 5-Fold Training (Base + LongSwap + Fixer) =====")

    for fold, (tr_idx, va_idx) in enumerate(gkf.split(X_train, y_dx, groups=groups), 1):
        print(f"\n--- Fold {fold} ---")

        X_tr = X_train.iloc[tr_idx]
        X_va = X_train.iloc[va_idx]

        # 1) Base
        model_dx = CatBoostRegressor(**Config.BASE_PARAMS)
        model_dy = CatBoostRegressor(**Config.BASE_PARAMS)

        model_dx.fit(
            X_tr, y_dx[tr_idx],
            eval_set=[(X_va, y_dx[va_idx])],
            cat_features=cat_features, verbose=verbose
        )
        model_dy.fit(
            X_tr, y_dy[tr_idx],
            eval_set=[(X_va, y_dy[va_idx])],
            cat_features=cat_features, verbose=verbose
        )

        # 2) long 마스크(Train/Val은 true dist)
        tr_dist = np.sqrt(y_dx[tr_idx]**2 + y_dy[tr_idx]**2)
        va_dist = np.sqrt(y_dx[va_idx]**2 + y_dy[va_idx]**2)

        mask_long_tr = (tr_dist >= long_threshold)
        mask_long_va = (va_dist >= long_threshold)

        if mask_long_tr.sum() >= min_long_samples:
            long_dx_model, long_dy_model = train_long_model(
                X_tr[mask_long_tr],
                y_dx[tr_idx][mask_long_tr],
                y_dy[tr_idx][mask_long_tr],
                cat_features
            )
        else:
            long_dx_model, long_dy_model = None, None
            print(f"[Fold {fold}] long samples 부족({mask_long_tr.sum()}), long swap 스킵")

        # 3) OOF base pred
        val_dx = model_dx.predict(X_va).astype(np.float32)
        val_dy = model_dy.predict(X_va).astype(np.float32)

        # 4) OOF long swap (val은 true dist 기준)
        if long_dx_model is not None and mask_long_va.sum() > 0:
            val_dx[mask_long_va] = long_dx_model.predict(X_va[mask_long_va]).astype(np.float32)
            val_dy[mask_long_va] = long_dy_model.predict(X_va[mask_long_va]).astype(np.float32)

        # 5) OOF 저장
        oof_base_dx[va_idx] = val_dx
        oof_base_dy[va_idx] = val_dy

        # 6) TEST base pred + TEST long swap(예측거리 기준)
        test_dx_fold = model_dx.predict(X_test).astype(np.float32)
        test_dy_fold = model_dy.predict(X_test).astype(np.float32)

        if long_dx_model is not None:
            test_dist_pred = np.sqrt(test_dx_fold**2 + test_dy_fold**2)
            mask_long_te = (test_dist_pred >= long_threshold)
            if mask_long_te.sum() > 0:
                test_dx_fold[mask_long_te] = long_dx_model.predict(X_test[mask_long_te]).astype(np.float32)
                test_dy_fold[mask_long_te] = long_dy_model.predict(X_test[mask_long_te]).astype(np.float32)

        test_base_dx += test_dx_fold / Config.N_FOLDS
        test_base_dy += test_dy_fold / Config.N_FOLDS

        # 중요도
        fi_df["importance"] += model_dx.get_feature_importance() / Config.N_FOLDS

        # 7) Fixer (잔차)
        train_dx_pred = model_dx.predict(X_tr).astype(np.float32)
        train_dy_pred = model_dy.predict(X_tr).astype(np.float32)

        resid_dx_tr = y_dx[tr_idx] - train_dx_pred
        resid_dy_tr = y_dy[tr_idx] - train_dy_pred

        resid_dx_va = y_dx[va_idx] - val_dx
        resid_dy_va = y_dy[va_idx] - val_dy

        fixer_dx = CatBoostRegressor(**Config.FIXER_PARAMS)
        fixer_dy = CatBoostRegressor(**Config.FIXER_PARAMS)

        fixer_dx.fit(
            X_tr, resid_dx_tr,
            eval_set=[(X_va, resid_dx_va)],
            cat_features=cat_features, verbose=verbose
        )
        fixer_dy.fit(
            X_tr, resid_dy_tr,
            eval_set=[(X_va, resid_dy_va)],
            cat_features=cat_features, verbose=verbose
        )

        oof_fix_dx[va_idx] = fixer_dx.predict(X_va).astype(np.float32)
        oof_fix_dy[va_idx] = fixer_dy.predict(X_va).astype(np.float32)

        test_fix_dx += fixer_dx.predict(X_test).astype(np.float32) / Config.N_FOLDS
        test_fix_dy += fixer_dy.predict(X_test).astype(np.float32) / Config.N_FOLDS

    # 최종 합
    oof_pred_dx = oof_base_dx + oof_fix_dx
    oof_pred_dy = oof_base_dy + oof_fix_dy

    test_pred_dx = test_base_dx + test_fix_dx
    test_pred_dy = test_base_dy + test_fix_dy

    oof_df = X_train.copy()
    oof_df["true_dx"] = y_dx
    oof_df["true_dy"] = y_dy
    oof_df["pred_dx"] = oof_pred_dx
    oof_df["pred_dy"] = oof_pred_dy
    oof_df["pass_dist"] = np.sqrt(oof_df["true_dx"]**2 + oof_df["true_dy"]**2)

    score = metric(y_dx, y_dy, oof_pred_dx, oof_pred_dy)
    print(f"\n[CatBoost] OOF: {score:.4f}m")

    return oof_pred_dx, oof_pred_dy, test_pred_dx, test_pred_dy, fi_df, oof_df

In [ ]:
# 1) (LGBM) 인코딩: 범주형은 __NA__로 처리 후 factorize
#    수치형만 -999로 결측 처리

def encode_for_lgbm(X_train, X_test, cat_cols, make_category_dtype=True):
    Xtr = X_train.copy()
    Xte = X_test.copy()
    ntr = len(Xtr)

    cat_cols = [c for c in cat_cols if c in Xtr.columns]

    # 범주형: __NA__ -> factorize (train+test together)
    for c in cat_cols:
        combined = pd.concat([Xtr[c], Xte[c]], axis=0).astype("object").fillna("__NA__")
        codes, _ = pd.factorize(combined, sort=False)

        Xtr[c] = pd.Series(codes[:ntr], index=Xtr.index).astype("int32")
        Xte[c] = pd.Series(codes[ntr:], index=Xte.index).astype("int32")

        if make_category_dtype:
            Xtr[c] = Xtr[c].astype("category")
            Xte[c] = Xte[c].astype("category")

    # 수치형: inf/NaN -> -999
    num_cols = [c for c in Xtr.columns if c not in cat_cols]
    if num_cols:
        Xtr[num_cols] = Xtr[num_cols].replace([np.inf, -np.inf], np.nan).fillna(-999)
        Xte[num_cols] = Xte[num_cols].replace([np.inf, -np.inf], np.nan).fillna(-999)

    return Xtr, Xte

In [ ]:
import lightgbm as lgb

# 3) LightGBM: A1 잔차 Fixer 2-stage + A2 LongSwap
def run_lgbm_oof_residual_longswap(
    X_train, y_dx, y_dy, X_test, groups, cat_cols,
    long_threshold=35.0, min_long_samples=50,
    make_category_dtype=True,
    base_params=None, fixer_params=None,
    num_boost_round=6000, early_stopping_rounds=200, verbose_eval=200
):
    Xtr, Xte = encode_for_lgbm(X_train, X_test, cat_cols, make_category_dtype=make_category_dtype)
    gkf = GroupKFold(n_splits=Config.N_FOLDS)
    cat_feats_present = [c for c in cat_cols if c in Xtr.columns]


    # 기본 파라미터
    if base_params is None:
        base_params = dict(
            objective="huber",
            alpha=0.9,
            metric="rmse",
            learning_rate=0.03,
            num_leaves=64,
            feature_fraction=0.8,
            bagging_fraction=0.8,
            bagging_freq=1,
            min_data_in_leaf=50,
            lambda_l2=5.0,
            seed=Config.SEED if hasattr(Config, "SEED") else 42,
            verbosity=-1
        )
    if fixer_params is None:
        fixer_params = dict(
            objective="regression_l1",  # 잔차는 L1 안정적
            metric="rmse",
            learning_rate=0.03,
            num_leaves=48,
            feature_fraction=0.8,
            bagging_fraction=0.8,
            bagging_freq=1,
            min_data_in_leaf=80,
            lambda_l2=8.0,
            seed=Config.SEED if hasattr(Config, "SEED") else 42,
            verbosity=-1
        )

    # OOF/TEST 저장소
    oof_base_dx = np.zeros(len(Xtr), dtype=np.float32)
    oof_base_dy = np.zeros(len(Xtr), dtype=np.float32)
    oof_fix_dx  = np.zeros(len(Xtr), dtype=np.float32)
    oof_fix_dy  = np.zeros(len(Xtr), dtype=np.float32)

    test_base_dx = np.zeros(len(Xte), dtype=np.float32)
    test_base_dy = np.zeros(len(Xte), dtype=np.float32)
    test_fix_dx  = np.zeros(len(Xte), dtype=np.float32)
    test_fix_dy  = np.zeros(len(Xte), dtype=np.float32)

    print("\n===== [LightGBM] 5-Fold Training (Base + Fixer + LongSwap) =====")

    for fold, (tr_idx, va_idx) in enumerate(gkf.split(Xtr, y_dx, groups=groups), 1):
        print(f"\n[LGBM] Fold {fold}")

        X_tr, X_va = Xtr.iloc[tr_idx], Xtr.iloc[va_idx]
        ydx_tr, ydx_va = y_dx[tr_idx], y_dx[va_idx]
        ydy_tr, ydy_va = y_dy[tr_idx], y_dy[va_idx]

        # ---------- Base dx ----------
        dtrain_dx = lgb.Dataset(X_tr, label=ydx_tr, categorical_feature=cat_feats_present, free_raw_data=False)
        dvalid_dx = lgb.Dataset(X_va, label=ydx_va, categorical_feature=cat_feats_present, free_raw_data=False)

        base_dx = lgb.train(
            base_params, dtrain_dx, num_boost_round=num_boost_round, valid_sets=[dvalid_dx],
            callbacks=[lgb.early_stopping(early_stopping_rounds, verbose=True),
                       lgb.log_evaluation(period=verbose_eval)]
        )

        # ---------- Base dy ----------
        dtrain_dy = lgb.Dataset(X_tr, label=ydy_tr, categorical_feature=cat_feats_present, free_raw_data=False)
        dvalid_dy = lgb.Dataset(X_va, label=ydy_va, categorical_feature=cat_feats_present, free_raw_data=False)

        base_dy = lgb.train(
            base_params, dtrain_dy, num_boost_round=num_boost_round, valid_sets=[dvalid_dy],
            callbacks=[lgb.early_stopping(early_stopping_rounds, verbose=True),
                       lgb.log_evaluation(period=verbose_eval)]
        )

        # base pred (val)
        val_base_dx = base_dx.predict(X_va, num_iteration=base_dx.best_iteration).astype(np.float32)
        val_base_dy = base_dy.predict(X_va, num_iteration=base_dy.best_iteration).astype(np.float32)
        oof_base_dx[va_idx] = val_base_dx
        oof_base_dy[va_idx] = val_base_dy

        # base pred (test fold)
        te_base_dx = base_dx.predict(Xte, num_iteration=base_dx.best_iteration).astype(np.float32)
        te_base_dy = base_dy.predict(Xte, num_iteration=base_dy.best_iteration).astype(np.float32)
        test_base_dx += te_base_dx / Config.N_FOLDS
        test_base_dy += te_base_dy / Config.N_FOLDS

        # ---------- Fixer residual targets ----------
        tr_base_dx = base_dx.predict(X_tr, num_iteration=base_dx.best_iteration).astype(np.float32)
        tr_base_dy = base_dy.predict(X_tr, num_iteration=base_dy.best_iteration).astype(np.float32)

        resid_dx_tr = (ydx_tr - tr_base_dx).astype(np.float32)
        resid_dy_tr = (ydy_tr - tr_base_dy).astype(np.float32)

        resid_dx_va_true = (ydx_va - val_base_dx).astype(np.float32)
        resid_dy_va_true = (ydy_va - val_base_dy).astype(np.float32)

        # ---------- Fixer dx ----------
        ftrain_dx = lgb.Dataset(X_tr, label=resid_dx_tr, categorical_feature=cat_feats_present, free_raw_data=False)
        fvalid_dx = lgb.Dataset(X_va, label=resid_dx_va_true, categorical_feature=cat_feats_present, free_raw_data=False)

        fix_dx = lgb.train(
            fixer_params, ftrain_dx, num_boost_round=num_boost_round, valid_sets=[fvalid_dx],
            callbacks=[lgb.early_stopping(early_stopping_rounds, verbose=True),
                       lgb.log_evaluation(period=verbose_eval)]
        )

        # ---------- Fixer dy ----------
        ftrain_dy = lgb.Dataset(X_tr, label=resid_dy_tr, categorical_feature=cat_feats_present, free_raw_data=False)
        fvalid_dy = lgb.Dataset(X_va, label=resid_dy_va_true, categorical_feature=cat_feats_present, free_raw_data=False)

        fix_dy = lgb.train(
            fixer_params, ftrain_dy, num_boost_round=num_boost_round, valid_sets=[fvalid_dy],
            callbacks=[lgb.early_stopping(early_stopping_rounds, verbose=True),
                       lgb.log_evaluation(period=verbose_eval)]
        )

        # fixer pred
        val_fix_dx = fix_dx.predict(X_va, num_iteration=fix_dx.best_iteration).astype(np.float32)
        val_fix_dy = fix_dy.predict(X_va, num_iteration=fix_dy.best_iteration).astype(np.float32)
        oof_fix_dx[va_idx] = val_fix_dx
        oof_fix_dy[va_idx] = val_fix_dy

        te_fix_dx = fix_dx.predict(Xte, num_iteration=fix_dx.best_iteration).astype(np.float32)
        te_fix_dy = fix_dy.predict(Xte, num_iteration=fix_dy.best_iteration).astype(np.float32)
        test_fix_dx += te_fix_dx / Config.N_FOLDS
        test_fix_dy += te_fix_dy / Config.N_FOLDS

    # base+fixer 합
    oof_dx = oof_base_dx + oof_fix_dx
    oof_dy = oof_base_dy + oof_fix_dy
    test_dx = test_base_dx + test_fix_dx
    test_dy = test_base_dy + test_fix_dy

    # ---------- A2) LongSwap ----------
    # OOF: true dist 기준으로 long 구간 swap (train_epi의 true dist 필요)
    true_dist = np.sqrt(y_dx**2 + y_dy**2)
    mask_long_oof = (true_dist >= long_threshold)

    # long 모델 학습은 "전체 train의 long"으로 한번 실행
    if mask_long_oof.sum() >= min_long_samples:
        # long 전용은 base+fixer가 아닌, "별도 long 전용 base"로 가는 게 보통 더 안정적
        # 여기서는 간단히 long 전용 dx/dy를 따로 학습
        # (다만 cat_cols는 이미 인코딩된 Xtr/Xte를 쓰므로 cat_feats_present 그대로)
        import lightgbm as lgb

        X_long = Xtr[mask_long_oof]
        ydx_long = y_dx[mask_long_oof]
        ydy_long = y_dy[mask_long_oof]

        dlong_dx = lgb.Dataset(X_long, label=ydx_long, categorical_feature=cat_feats_present, free_raw_data=False)
        dlong_dy = lgb.Dataset(X_long, label=ydy_long, categorical_feature=cat_feats_present, free_raw_data=False)

        long_params = dict(base_params)
        long_params.update({"learning_rate": 0.05, "num_leaves": 96, "min_data_in_leaf": 30})

        long_dx = lgb.train(long_params, dlong_dx, num_boost_round=1500, valid_sets=None)
        long_dy = lgb.train(long_params, dlong_dy, num_boost_round=1500, valid_sets=None)

        # OOF long swap (true dist mask)
        if mask_long_oof.sum() > 0:
            oof_dx_swap = oof_dx.copy()
            oof_dy_swap = oof_dy.copy()
            oof_dx_swap[mask_long_oof] = long_dx.predict(Xtr.iloc[np.where(mask_long_oof)[0]]).astype(np.float32)
            oof_dy_swap[mask_long_oof] = long_dy.predict(Xtr.iloc[np.where(mask_long_oof)[0]]).astype(np.float32)
            oof_dx, oof_dy = oof_dx_swap, oof_dy_swap

        # TEST long swap (pred dist 기준)
        pred_dist_test = np.sqrt(test_dx**2 + test_dy**2)
        mask_long_te = (pred_dist_test >= long_threshold)
        if mask_long_te.sum() > 0:
            test_dx_swap = test_dx.copy()
            test_dy_swap = test_dy.copy()
            test_dx_swap[mask_long_te] = long_dx.predict(Xte.iloc[np.where(mask_long_te)[0]]).astype(np.float32)
            test_dy_swap[mask_long_te] = long_dy.predict(Xte.iloc[np.where(mask_long_te)[0]]).astype(np.float32)
            test_dx, test_dy = test_dx_swap, test_dy_swap

    oof_score = metric(y_dx, y_dy, oof_dx, oof_dy)
    print(f"\n[LightGBM] OOF: {oof_score:.4f}m")

    return oof_dx, oof_dy, test_dx, test_dy

In [ ]:
# 1) MLP 구조 정의
class PassMLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(256, 128), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 2)  # dx, dy
        )

    def forward(self, x):
        return self.net(x)


# 2) 유클리디안 Loss (L2 거리 직접 최적화)
class EuclideanLoss(nn.Module):
    def forward(self, pred, target):
        return torch.mean(torch.sqrt(torch.sum((pred - target) ** 2, dim=1) + 1e-7))


# 3) 학습 함수 (OOF/TEST 생성)
def run_mlp_pipeline(
    X_train, y_dx, y_dy, X_test, groups, cat_cols,
    n_splits=5, epochs=60, batch_size=256, lr=0.002,
    seed=42, save_dir="models", device=None
):
    # ----- 재현성 -----
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    # ----- device -----
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    # ----- 폴더 생성 (저장 에러 방지) -----
    os.makedirs(save_dir, exist_ok=True)

    # [수정] 범주형 피처를 숫자로 변환 (Train/Test 합쳐서 factorize)
    Xtr = X_train.copy()
    Xte = X_test.copy()

    for c in cat_cols:
        if c in Xtr.columns:
            combined = pd.concat([Xtr[c], Xte[c]], axis=0).astype("object").fillna("__NA__")
            codes, _ = pd.factorize(combined, sort=False)
            Xtr[c] = codes[:len(Xtr)].astype(np.int64)
            Xte[c] = codes[len(Xtr):].astype(np.int64)

    # StandardScaler
    scaler = StandardScaler()
    Xtr_s = scaler.fit_transform(Xtr.fillna(0))
    Xte_s = scaler.transform(Xte.fillna(0))

    # OOF/TEST 저장소
    oof_dx = np.zeros(len(Xtr_s), dtype=np.float32)
    oof_dy = np.zeros(len(Xtr_s), dtype=np.float32)
    test_dx = np.zeros(len(Xte_s), dtype=np.float32)
    test_dy = np.zeros(len(Xte_s), dtype=np.float32)

    gkf = GroupKFold(n_splits=n_splits)

    for fold, (tr_idx, va_idx) in enumerate(gkf.split(Xtr_s, y_dx, groups=groups), 1):
        # dataset/loader
        X_tr = torch.tensor(Xtr_s[tr_idx], dtype=torch.float32)
        y_tr = torch.tensor(np.column_stack([y_dx[tr_idx], y_dy[tr_idx]]), dtype=torch.float32)
        train_ds = TensorDataset(X_tr, y_tr)
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        # model
        model = PassMLP(Xtr_s.shape[1]).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        criterion = EuclideanLoss()

        # train
        model.train()
        for epoch in range(epochs):
            for bx, by in train_loader:
                bx, by = bx.to(device), by.to(device)
                optimizer.zero_grad()
                pred = model(bx)
                loss = criterion(pred, by)
                loss.backward()
                optimizer.step()

        # 모델 저장
        torch.save(model.state_dict(), os.path.join(save_dir, f"mlp_fold_{fold}.pt"))

        # eval
        model.eval()
        with torch.no_grad():
            X_va = torch.tensor(Xtr_s[va_idx], dtype=torch.float32).to(device)
            val_p = model(X_va).detach().cpu().numpy()
            oof_dx[va_idx], oof_dy[va_idx] = val_p[:, 0], val_p[:, 1]

            X_te = torch.tensor(Xte_s, dtype=torch.float32).to(device)
            te_p = model(X_te).detach().cpu().numpy()
            test_dx += te_p[:, 0] / n_splits
            test_dy += te_p[:, 1] / n_splits

        print(f"MLP Fold {fold} Done")

    return oof_dx, oof_dy, test_dx, test_dy

In [ ]:
X_train, X_test, y_dx, y_dy, groups, test_epi, cat_features = build_dataset(df_copy)

def run_catboost_pipeline(
    X_train, y_dx, y_dy, X_test, groups, cat_features,
    long_threshold=35.0, min_long_samples=50, verbose=200
):
    cb_oof_dx, cb_oof_dy, cb_test_dx, cb_test_dy, cb_fi, cb_oof_df = train_residual_pipeline_cat(
        X_train, y_dx, y_dy, X_test, groups, cat_features,
        long_threshold=long_threshold,
        min_long_samples=min_long_samples,
        verbose=verbose
    )
    return cb_oof_dx, cb_oof_dy, cb_test_dx, cb_test_dy, cb_oof_df, cb_fi

In [ ]:
# --- CatBoost ---
cb_oof_dx, cb_oof_dy, cb_test_dx, cb_test_dy, cb_oof_df, cb_fi = run_catboost_pipeline(
    X_train, y_dx, y_dy, X_test, groups, cat_features,
    long_threshold=35.0, min_long_samples=50, verbose=200
)


===== [CatBoost] 5-Fold Training (Base + LongSwap + Fixer) =====

--- Fold 1 ---
0:	learn: 16.0111154	test: 15.9181020	best: 15.9181020 (0)	total: 96.5ms	remaining: 9m 38s
200:	learn: 10.9461143	test: 11.6283953	best: 11.6283953 (200)	total: 20s	remaining: 9m 35s
400:	learn: 10.2273040	test: 11.4526273	best: 11.4521784 (399)	total: 41.2s	remaining: 9m 35s
600:	learn: 9.5259459	test: 11.3788610	best: 11.3788610 (600)	total: 1m 1s	remaining: 9m 10s
800:	learn: 8.9523175	test: 11.3588203	best: 11.3554834 (773)	total: 1m 25s	remaining: 9m 12s
1000:	learn: 8.3748086	test: 11.3537590	best: 11.3503561 (914)	total: 2m 5s	remaining: 10m 29s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 11.3503561
bestIteration = 914

Shrink model to first 915 iterations.
0:	learn: 14.9576729	test: 14.7510436	best: 14.7510436 (0)	total: 221ms	remaining: 22m 7s
200:	learn: 11.7135582	test: 12.4239115	best: 12.4239115 (200)	total: 34.9s	remaining: 16m 47s
400:	learn: 10.9114416	test: 12.24402

In [ ]:
# --- LightGBM (A1 + A2) ---
lgb_oof_dx, lgb_oof_dy, lgb_test_dx, lgb_test_dy = run_lgbm_oof_residual_longswap(
    X_train, y_dx, y_dy, X_test, groups, cat_cols=cat_features,
    long_threshold=35.0, min_long_samples=200,
    make_category_dtype=True,
    base_params=None, fixer_params=None,
    num_boost_round=6000, early_stopping_rounds=200, verbose_eval=200
)


===== [LightGBM] 5-Fold Training (Base + Fixer + LongSwap) =====

[LGBM] Fold 1
Training until validation scores don't improve for 200 rounds
[200]	valid_0's rmse: 14.6073
[400]	valid_0's rmse: 13.6277
[600]	valid_0's rmse: 12.9871
[800]	valid_0's rmse: 12.5689
[1000]	valid_0's rmse: 12.2885
[1200]	valid_0's rmse: 12.0957
[1400]	valid_0's rmse: 11.9552
[1600]	valid_0's rmse: 11.8516
[1800]	valid_0's rmse: 11.7766
[2000]	valid_0's rmse: 11.7279
[2200]	valid_0's rmse: 11.6963
[2400]	valid_0's rmse: 11.6764
[2600]	valid_0's rmse: 11.6608
[2800]	valid_0's rmse: 11.6502
[3000]	valid_0's rmse: 11.6424
[3200]	valid_0's rmse: 11.6346
[3400]	valid_0's rmse: 11.6293
[3600]	valid_0's rmse: 11.6228
[3800]	valid_0's rmse: 11.6187
[4000]	valid_0's rmse: 11.6156
[4200]	valid_0's rmse: 11.6126
[4400]	valid_0's rmse: 11.6087
[4600]	valid_0's rmse: 11.6068
[4800]	valid_0's rmse: 11.6025
[5000]	valid_0's rmse: 11.6003
[5200]	valid_0's rmse: 11.6001
Early stopping, best iteration is:
[5073]	valid_0's rms

In [ ]:
mlp_oof_dx, mlp_oof_dy, mlp_test_dx, mlp_test_dy = run_mlp_pipeline(
    X_train, y_dx, y_dy, X_test, groups, cat_features
)

MLP Fold 1 Done
MLP Fold 2 Done
MLP Fold 3 Done
MLP Fold 4 Done
MLP Fold 5 Done


In [ ]:
# MLP 단독 OOF 점수 출력
mlp_score = metric(y_dx, y_dy, mlp_oof_dx, mlp_oof_dy)
print(f"✨ MLP 단독 OOF 점수: {mlp_score:.4f}m")

# 기존 부스팅 모델들과 비교
print(f"  - CatBoost : {cb_score_all:.4f}m")
print(f"  - LightGBM : {lg_score_all:.4f}m")

✨ MLP 단독 OOF 점수: 13.3885m
  - CatBoost : 13.0940m
  - LightGBM : 13.0824m


In [ ]:
# 3중 버킷 가중치 결합 및 결과 확인

# 1. 3개 모델의 OOF/TEST 데이터를 튜플로 묶기
cb_res  = (cb_oof_dx, cb_oof_dy, cb_test_dx, cb_test_dy)
lgb_res = (lgb_oof_dx, lgb_oof_dy, lgb_test_dx, lgb_test_dy)
mlp_res = (mlp_oof_dx, mlp_oof_dy, mlp_test_dx, mlp_test_dy)

# 2. 3중 가중치 계산 (이미 정의한 함수 호출)
dist_true = np.sqrt(y_dx**2 + y_dy**2)
buckets_true = np.array([bucket_from_dist(d) for d in dist_true])
weights_triple = {}

for b in ["short", "mid", "long"]:
    idx = np.where(buckets_true == b)[0]
    # 각 구간별 RMSE 계산
    s_cb = metric(y_dx[idx], y_dy[idx], cb_res[0][idx], cb_res[1][idx])
    s_lg = metric(y_dx[idx], y_dy[idx], lgb_res[0][idx], lgb_res[1][idx])
    s_ml = metric(y_dx[idx], y_dy[idx], mlp_res[0][idx], mlp_res[1][idx])

    # 성능 기반 가중치 (지수 alpha=0.5 적용)
    inv_s = np.array([1/s_cb, 1/s_lg, 1/s_ml]) ** best_alpha

    weights_triple[b] = inv_s / inv_s.sum()

# 3. 가중치 출력 (MLP가 어느 구간에서 기여하는지 확인 가능)
print("\n[3중 앙상블 구간별 가중치 (CB, LGBM, MLP)]")
for b, w in weights_triple.items():
    print(f"  {b:5} : {w[0]:.4f}, {w[1]:.4f}, {w[2]:.4f}")

# 4. OOF 결합 및 최종 점수 확인
ens_oof_dx = np.zeros_like(y_dx)
ens_oof_dy = np.zeros_like(y_dy)

for b in ["short", "mid", "long"]:
    idx = np.where(buckets_true == b)[0]
    w_cb, w_lg, w_ml = weights_triple[b]
    ens_oof_dx[idx] = w_cb*cb_res[0][idx] + w_lg*lgb_res[0][idx] + w_ml*mlp_res[0][idx]
    ens_oof_dy[idx] = w_cb*cb_res[1][idx] + w_lg*lgb_res[1][idx] + w_ml*mlp_res[1][idx]

final_oof_score = metric(y_dx, y_dy, ens_oof_dx, ens_oof_dy)
print(f"\n [최종 3중 앙상블 OOF 점수] : {final_oof_score:.4f}m")
print(f"   (기존 2중 앙상블:12.98)")


[3중 앙상블 구간별 가중치 (CB, LGBM, MLP)]
  short : 0.1126, 0.3790, 0.5085
  mid   : 0.3480, 0.5410, 0.1110
  long  : 0.5514, 0.1961, 0.2525

🔥 [최종 3중 앙상블 OOF 점수] : 12.8054m
   (기존 2중 앙상블:12.98)


In [ ]:
#  3중 앙상블 적용 및 제출 파일 저장

# 1. 테스트 데이터의 버킷 기준 거리 생성 (가장 안정적인 CB 예측값 기준)
test_dist_ref = np.sqrt(cb_test_dx**2 + cb_test_dy**2)
buckets_test = np.array([bucket_from_dist(d) for d in test_dist_ref])

# 2. 결과 저장용 배열 초기화
ens_test_dx = np.zeros_like(cb_test_dx, dtype=np.float32)
ens_test_dy = np.zeros_like(cb_test_dy, dtype=np.float32)

# 3. 구간별 3중 가중치 적용 (CB, LGBM, MLP 순)
for b in ["short", "mid", "long"]:
    idx = np.where(buckets_test == b)[0]
    if len(idx) == 0: continue

    # 방금 출력된 황금 비율(weights_triple) 적용
    w_cb, w_lg, w_ml = weights_triple[b]

    ens_test_dx[idx] = (w_cb * cb_test_dx[idx] +
                        w_lg * lgb_test_dx[idx] +
                        w_ml * mlp_test_dx[idx])

    ens_test_dy[idx] = (w_cb * cb_test_dy[idx] +
                        w_lg * lgb_test_dy[idx] +
                        w_ml * mlp_test_dy[idx])

# 4. (선택) Carry Cap 적용 - 점수 향상이 검증되었다면 사용
if after < before:
    print("Applying Carry Cap to Test set...")
    ens_test_dx, ens_test_dy = apply_carry_hard_caps(ens_test_dx, ens_test_dy, test_epi, caps)

# 5. 최종 좌표 계산 및 Clipping
final_pred_x = np.clip(test_epi["start_x"].values + ens_test_dx, *Config.CLIP_X)
final_pred_y = np.clip(test_epi["start_y"].values + ens_test_dy, *Config.CLIP_Y)

# 6. 제출 파일 저장
submission = pd.DataFrame({
    "game_episode": test_epi["game_episode"].values,
    "end_x": final_pred_x,
    "end_y": final_pred_y
})

submission.to_csv("sub_triple_ensemble_final.csv", index=False)
print("\n✅ 제출 파일 저장 완료: sub_triple_ensemble_final.csv")


✅ 제출 파일 저장 완료: sub_triple_ensemble_final.csv
